<a href="https://colab.research.google.com/github/HeptaDecane/LP1_SEM7/blob/main/CudaSandbox/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-11-07 07:09:30--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-11-07 07:09:30--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?bjlY8zMpYsKBISDUHTIElkYo1FYlOfLKMSksd_ZUsTHM-KVXBILtAT44kxAx6MuazXqiOg8XWGGqS8nVkS63PREbJ0VO_nV6J5dJ-SvKp8_OZ

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [3]:
code = """
#include<iostream>
#include<math.h>

#define n 8

using namespace std;

__global__ void minimum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;

    printf("No of threads = %d", number_of_threads);

    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] < input[first])
                input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void maximum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;

    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] > input[first])
                input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void sum(int *input) {
    const int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;

    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;

            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;

    }
}

__global__ void mean_diff_sq(float *input, float mean) {
    input[threadIdx.x] -= mean;
    input[threadIdx.x] *= input[threadIdx.x];
}

__global__ void sum_floats(float *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;

    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;

            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;

    }
}

void copy_int_to_float(float *dest, int *src, int size){
    for(int i=0; i<size; i++)
        dest[i] = float(src[i]);
}

void random_ints(int *input, int size) {
    cout<<"Input: ";
    for(int i=0; i<size; i++)  {
        input[i] = rand()%100;
        cout<<input[i]<<"  ";
    }
    cout<<endl;

}

int main() {
    //int n=8;
    int size = n*sizeof(int); //calculate no. of bytes for array

    int *arr;
    int *arr_d, result;

    arr = (int *)malloc(size);
    random_ints(arr, n);

    cudaMalloc((void **)&arr_d, size);

    //MIN
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);

    minimum<<<1,n/2>>>(arr_d);

    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);

    cout<<"The minimum element is "<<result<<endl;


    //MAX
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);

    maximum<<<1,n/2>>>(arr_d);

    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);

    cout<<"The maximum element is "<<result<<endl;

    //SUM
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);

    sum<<<1,n/2>>>(arr_d);

    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);

    cout<<"The sum is "<<result<<endl;

    //AVERAGE

    float mean = float(result)/n;
    cout<<"The mean is "<<mean<<endl;

    //STANDARD DEVIATION
    float *arr_float;
    float *arr_std, stdValue;

    arr_float = (float *)malloc(n*sizeof(float));
    cudaMalloc((void **)&arr_std, n*sizeof(float));

    copy_int_to_float(arr_float, arr, n);

    cudaMemcpy(arr_std, arr_float, n*sizeof(float), cudaMemcpyHostToDevice);

    mean_diff_sq <<<1,n>>>(arr_std, mean);
    sum_floats<<<1,n/2>>>(arr_std);

    cudaMemcpy(&stdValue, arr_std, sizeof(float), cudaMemcpyDeviceToHost);


    stdValue = stdValue / n;
    cout<<"The variance is "<<stdValue<<endl;
    stdValue = sqrt(stdValue);

    cout<<"The standard deviation is "<<stdValue<<endl;

    cudaFree(arr_d);

    return 0;
}
"""

In [4]:
file = open("parallel_reduction.cu", "w")
file.write(code)
file.close()

In [5]:
!nvcc parallel_reduction.cu

In [6]:
!./a.out

Input: 83  86  77  15  93  35  86  92  
No of threads = 4No of threads = 4No of threads = 4No of threads = 4The minimum element is 15
The maximum element is 93
The sum is 567
The mean is 70.875
The variance is 748.359
The standard deviation is 27.3562


In [7]:
!nvprof ./a.out

Input: 83  86  77  15  93  35  86  92  
==1872== NVPROF is profiling process 1872, command: ./a.out
No of threads = 4No of threads = 4No of threads = 4No of threads = 4The minimum element is 15
The maximum element is 93
The sum is 567
The mean is 70.875
The variance is 748.359
The standard deviation is 27.3562
==1872== Profiling application: ./a.out
==1872== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   82.66%  150.59us         1  150.59us  150.59us  150.59us  minimum(int*)
                    4.97%  9.0560us         4  2.2640us  2.0480us  2.6240us  [CUDA memcpy DtoH]
                    3.74%  6.8160us         4  1.7040us  1.5040us  2.2720us  [CUDA memcpy HtoD]
                    2.60%  4.7360us         1  4.7360us  4.7360us  4.7360us  maximum(int*)
                    2.21%  4.0310us         1  4.0310us  4.0310us  4.0310us  sum_floats(float*)
                    2.13%  3.8720us         1  3.8720us  3.8720us  3.